# Atomate2 Workflow Ingestor 
###  Kat Nykiel, Alejandro Strachan
School of Materials Engineering and Birck Nanotechnology Center, Purdue University, West Lafayette, Indiana 47907, United States

## Load Atomate2 Workflow

This Sim2L stores atomate2 workflows by ingesting each firework individually. To keep track of the workflows, you can run the following commands to query your atomate2 database.

```python
from fireworks import LaunchPad
from jobflow import SETTINGS
import json

#  query the launchpad for a workflow
lp = LaunchPad.auto_load()
wf = lp.workflows.find_one({"nodes": 93701})
with open("workflow.json", "w") as f:
    json.dump(wf['links'], f, default=str)

# query the job store for the fireworks in the given workflow
store = SETTINGS.JOB_STORE
store.connect()
fw_docs = list(store.query({"metadata.fw_id": {"$in": wf['nodes']}}))
with open("fireworks.json", "w") as f:
    json.dump(fw_docs, f, default=str)
```

In [ ]:
# Import libraries
import numpy as np
import json
import os
import pandas as pd

# Import nanoHUB-specific libraries
import nanohubremote as nr
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

Here we load an example elastic constant workflow. If you want to upload your own file, change the path to your chosen JSON files of the workflow graph and firework docs

In [ ]:
# Load json files
wf_path = './../examples/workflow.json' # workflow graph JSON file
with open(wf_path,'r') as f:
    wf_graph = json.load(f)   
    
fws_path = './../examples/fireworks.json' # firework docs JSON file
with open(fws_path,'r') as f:
    fw_docs = json.load(f)

## Load Sim2L

In [ ]:
# Load the Sim2L
simToolName = "wflowingestor"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print("%18s = %s" % (key,simToolLocation[key]))

installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

In [ ]:
# Get the list of inputs and outputs
inputs = getSimToolInputs(simToolLocation)
outputs = getSimToolOutputs(simToolLocation)

## Submit Sim2L sequentially

In [ ]:
# Set the inputs for the Sim2L
inputs['wf_graph'].value = wf_graph
inputs['author'].value = "Jane Doe"
inputs['dataset'].value = "example"

Here, we loop through the firework docs to ingest each in our resultsDB

In [ ]:
for fw_doc in fw_docs:
    
    inputs['fw_doc'].value = fw_doc
    r = Run(simToolLocation,inputs)